# Trainieren des neuronalen Netzes Kontinuierlich

Zu Beginn müssen wieder alle Bibliotheken eingebunden werden.

In [8]:
#import all necessary packages
import os
import csv
import numpy as np
import pandas as pd
#from numpy import genfromtxt
#from random import randint
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras

#%matplotlib inline
from sklearn.metrics import confusion_matrix
import itertools
import matplotlib.pyplot as plt

import IPython
import kerastuner as kt



Für den Fall, dass die bereits mit Skript 1 vorbereiteten Datensätze nicht im selben Verzeichnis wie die Python-Datei liegt muss das Verzeichnis des Datensatzes angegeben werden. Dies kann durch folgenden Befehl durchgeführt werden.

In [9]:
#function to set the start working directory manually
#not necessary, if the .py file is executed in the directory of the .csv files
#os.chdir('D:\\OneDrive - bwedu\\Uni\\09 ABC 1\\Neuronale Netzwerke\\Python\\Wein')



Für komplexere Datensätze oder variierte Trainingsparameter kann die Einbindung der GPU von Vorteil sein (für dieses Beispiel nicht benötigt). 
Die GPU kann ggf. durch folgende Codezeilen eingebunden werden:

In [10]:
#activation of the GPU, for this model not necessary
#physical_devices = tf.config.experimental.list_physical_devices('GPU')
#print("Num GPUs Available: ", len(physical_devices))
#tf.config.experimental.set_memory_growth(physical_devices[0], True)



Nun wird das Arbeitsverzeichnis so geändert, dass auf die zuvor vorbereiteten Daten zugegriffen werden kann. 

Codeabsatz 1 speichert das aktuelle Arbeitsverzeichnis.

Codeabsatz 2 ändert das Arbeitsverzeichnis.

Codeabsatz 3 importiert die mittels Datenvorbereitungsskript zuvor erstellten Daten.

Codeabsatz 4 convertiert die Daten in ein numpy-Array uns skaliert die daten in den Bereich von 0-1. 

Dabei ist zu beachten, dass die erste Spalte des Datensatzes nicht von Relevanz ist und daher nicht beachtet wird (erste Spalte dient nur zur Nummerierung).

Die Daten werden skaliert, indem von den Daten zuert 3 subtrahiert wird, und anschließend mit 6 dividiert wird (z.B. subtrahiere 3 von der Qualität 3-9 = Qualität 0-6, dividiere Qualität 0-6 mit 6 =0-1))

In [11]:
#save the current working directory	 
current_wd = os.getcwd()

#change the working directory to the 'prep_dataset' directroy, where the prepared data are saved as .csv files
os.chdir('prep_dataset')

#import the prepared data as panda dataframes
W_train_samples = pd.read_csv('wine_train_samples.csv')
W_train_labels = pd.read_csv('wine_train_labels_cont.csv')

#convert panda dataframe to a numpy array
#leave out the first column, because this are the rownumernumbers
W_train_samples = pd.DataFrame.to_numpy(W_train_samples)[:,1:13]
W_train_labels = (pd.DataFrame.to_numpy(W_train_labels)[:,1]-3)/6 #scale the labels in a range form 0 to 1

Da die Daten nun importiert wurden, kann wieder zum ursprünglichen Arbeitsverzeichnis gewechselt werden. 

In [12]:
#change the working directory back to the original working directory
os.chdir(current_wd)	 
	 


Folgende Codezeilen erzeugen ein zufälliges neuronales Netz (model_builder), welches später trainiert werden kann. 

Zu beachten ist, dass dem model_builder die Parameter (Anzahl layer, learning Rate und Anzahl Knoten) systematisch auswählt.

Für dieses Beispiel kann der model_builder die Parameter in folgenden Bereichen wählen:

Anzahl hidden layer: 1-5

Learning Rate: 1e-5 bis 1e-2

Knoten der hidden Layer: 16-160 (Differenz der Anzahl an Knoten zwischen den einzelnen Layer: 16 Knoten, um Zeit zu sparen)

Aktivierungsfunktion: relu oder sigmoid




z.B: 
Modell 1: mit 3 hidden layer, der Lernrate 1e-5 und 16, 32 und 48 Knoten jeweils mit der Aktivierungsfunktion relu
Modell 2: mit 2 hidden layer, der Lernrate 1e-3 und 64, 80 Knoten jeweils mit der Aktivierungsfunktion relu
Modell 3: mit 5 hidden layer, der Lernrate 1e-2 und 96, 112, 128, 144 und 160 Knoten jeweils mit der Aktivierungsfunktion relu
...

x repräsentiert jeweils das vorherige Modell 


Beim kompilieren des Modells ist zu beachten, dass hier als Fehler der mean square error berechnet wird. 
(Der mittlere quadratische Fehler (Mean Squared Error, MSE) eines Schätzers misst den Durchschnitt der Fehlerquadrate, d.h. die mittlere quadratische Differenz zwischen den geschätzten Werten und dem wahren Wert.) 


Zum Schluss wird das fertige Modell zurückgegeben. 

In [13]:
#the definition of the model building function
#here it is possible to specify the model, the most parameters
#with hp funcitions it is possible to specify a range of values, in which an optimizer can optimize the model
def model_builder(hp):
	hp_layers = hp.Int('layers', min_value = 1, max_value = 5)#the number of layers in a range from 1 to 5
	hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4, 1e-5]) #test different learning rates 
	
	#the input layer with 12 nodes
	inputs = keras.Input(shape=(12,))
	
	x = inputs
	for i in range(hp_layers):
		#for each layer a different number of nodes (16 to 160)
		x = keras.layers.Dense(units = hp.Int('units_' + str(i), min_value = 16, max_value = 160, step = 16), activation = hp.Choice('act_func_' + str(i), values = ["relu", "sigmoid"]))(x)
	
	#the output layer with 1 node (continous regression)
	outputs = keras.layers.Dense(1, activation = hp.Choice('act_func_output', values = ["relu", "sigmoid"]))(x)
	
	#set the model together
	model = keras.Model(inputs, outputs)
	
	#compile the model
	model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate), 
					loss = "mean_squared_error",
					metrics = ['mse'])
	
	return model #return the finished model





Nun werden die einzelnen Einstellungen für den tuner definiert. 

Wichtig: Sollte eine erneute Optimierung erfolgen, so muss vor jeder Wiederholung zuerst das 
Verzeichnis des vorherigen tunings manuell gelöscht werden.
    

In [14]:
#set settings for the tuner
tuner = kt.Hyperband(model_builder,
                     objective = 'val_mse', 
                     max_epochs = 200,
                     factor = 5, #the higher, the faster the optimizing, but the smalles the probability to find the best model
							#!!!delete directory before optimizing again!!!
                     directory = 'tuner_wine_continuous_actfunc',
                     project_name = 'wine_quality_continuous')





INFO:tensorflow:Reloading Oracle from existing project tuner_wine_continuous_actfunc\wine_quality_continuous\oracle.json
INFO:tensorflow:Reloading Tuner from tuner_wine_continuous_actfunc\wine_quality_continuous\tuner0.json


In unserem Code verwenden wir sogenannte Callback-Funktionen. 

Sobald über 5 Epochen (patience) kein Fortschritt (Verbesserung der Genauigkeit) mehr erreicht wird, wird das Training abgebrochen. 

In [15]:
#define the necessary callbacks 
early_stopping_cb = keras.callbacks.EarlyStopping(patience=5, restore_best_weights=True)



Nun werden die erzeugten neuronalen Netze optimiert. 
Folgende Parameter werden dafür verwendet:
    
    epochs: 200 -> Insgesamt maximal 200 Wiederholungen
    batch_size: 6 -> Paralleses Training des neuronalen Netzes durch Daten
    validation_split: 0.1 -> 10% der Daten werden für eine Validation verwendet
    callbacks (Abruf wie oben definiert)

In [16]:
#start the optimizer 
tuner.search(W_train_samples, W_train_labels, epochs = 200, batch_size = 6, validation_split = 0.1, callbacks = [early_stopping_cb])



INFO:tensorflow:Oracle triggered exit


Durch folgenden Befehl wird aus allen getesteten Parametersätzen der beste Parametersatz ausgewählt und gespeichert. 
Alle anderen neuronalen Netze werden nicht beachtet.

In [17]:
# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]



Durch folgenden print-Befehl werden die vorher definierten Parameter des besten neuronalen Netzes am Bildschirm ausgegeben. 

In [18]:
#print the results of the optimizer
print(f"""
		The hyperparameter search is complete.
		Learning rate: {best_hps.get('learning_rate')}
		Number of Layers: {best_hps.get('layers')}
		""")
for i in range(best_hps.get('layers')):
	print("Number of nodes in layer %i: %f" %(i, best_hps.get('units_' + str(i))))
	print("Activationfunction in layer %i:" %(i) + best_hps.get('act_func_' + str(i)) )
print("Activationfunction in outputlayer" + best_hps.get('act_func_output') )






		The hyperparameter search is complete.
		Learning rate: 0.001
		Number of Layers: 3
		
Number of nodes in layer 0: 112.000000
Activationfunction in layer 0:relu
Number of nodes in layer 1: 16.000000
Activationfunction in layer 1:relu
Number of nodes in layer 2: 96.000000
Activationfunction in layer 2:sigmoid
Activationfunction in outputlayersigmoid


Durch folgenden Befehl kann der Fortschritt graphisch in tensorboard dargestellt werden
(um die Daten graphisch darstellen zu können muss zuerst ein log-File erstellt werden).

In [19]:
#define the log-directory for the logfiles for tensorboard
root_logdir = os.path.join(os.curdir, "tensorboard_wine_quality_continuous")
def get_run_logdir():
	import time
	run_id = 'wine_quality_continuous_' + time.strftime("run_%Y_%m_%d-%H_%M_%S")
	return os.path.join(root_logdir, run_id)
run_logdir = get_run_logdir() # e.g., './my_logs/run_2019_06_07-15_15_22'



Die callbacks werden wie folgt implementiert:

Codezeile 1: Speichern der Resultate in einem logfile für Tensorboard.

Codezeile 2: Das neuronale Netz speichert immer nur das Modell mit der höchsten Genauigkeit (als .h5-Datei).

Codezeile 3 Das neuronale Netz stoppt das Training, wenn nach 10 Wiederholungen keine weitere Erhöhung der Genauigkeit stattfindet.

In [20]:
#define the necessary callbacks
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
checkpoint_cb = keras.callbacks.ModelCheckpoint("wine_quality_continious_actfunc.h5", save_best_only=True)
early_stopping_cb2 = keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)




In folgendem Schritt wird das optimale Modell der Variable model zugewiesen, die nun das optimale untrainierte neuronale Netz repräsentiert, das anschließend noch trainiert werden muss.



In [21]:
# Build the model with the optimal hyperparameters 
model = tuner.hypermodel.build(best_hps)



Mit folgendem Befehl erfolgt das finale Training des neuronalen Netzes.
Mit der batch_size 6, maximal 1000 Epochen, 10% der Daten als Validierung und der callback-Funktionen.

In [22]:
#train the model
history = model.fit(W_train_samples, W_train_labels, batch_size = 6, epochs = 1000, validation_split = 0.1, callbacks = [checkpoint_cb, early_stopping_cb2, tensorboard_cb])	



Epoch 1/1000
  1/870 [..............................] - ETA: 0s - loss: 0.0092 - mse: 0.0092WARNING:tensorflow:From C:\Users\Tobia\anaconda3\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
870/870 [==============================] - 1s 1ms/step - loss: 0.0169 - mse: 0.0169 - val_loss: 0.0145 - val_mse: 0.0145
Epoch 2/1000
870/870 [==============================] - 1s 818us/step - loss: 0.0151 - mse: 0.0151 - val_loss: 0.0144 - val_mse: 0.0144
Epoch 3/1000
870/870 [==============================] - 1s 956us/step - loss: 0.0148 - mse: 0.0148 - val_loss: 0.0141 - val_mse: 0.0141
Epoch 4/1000
870/870 [==============================] - 1s 935us/step - loss: 0.0145 - mse: 0.0145 - val_loss: 0.0137 - val_mse: 0.0137
Epoch 5/1000
870/870 [==============================] - 1s 901us/step - loss: 0.0145 - mse: 0.014